In [58]:
# 2) 유튜브 공범 컨텐츠동영상 데이터
import pandas as pd


channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')

"각 데이터의 ‘ct’컬럼을 시간으로 인식할수 있게 datatype을 변경하고 video 데이터의 videoname의 각 value 마다 몇개의 데이터씩 가지고 있는지 확인하라"
channel.ct = pd.to_datetime(channel.ct)
video.ct = pd.to_datetime(video.ct)
#print(video.groupby(["videoname"])["videoname"].count())

"수집된 각 video의 가장 최신화 된 날짜의 viewcount값을 출력하라" # SQL과 같이 생각해볼것
# groupby를 사용하면 테이블을 새로 만들어야함
# sorting 하고, 중복되는 이름을 지우는데, 마지막 row 값만 남긴다, [[]] << select , orderby
answer = video.sort_values(by = ['videoname','ct']).drop_duplicates('videoname',keep='last')[['viewcnt','videoname','ct']].reset_index(drop=True)
# display(answer)

"Channel 데이터중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수(subcnt)를 출력하라"
# groupby 쓰지말고 하는 연습하기
answer = channel.loc[channel.ct >= pd.to_datetime("2021-10-03")] # datetime 데이터 타입은 형식 그냥 작성하면 알아서 계산해줌..
answer = answer.sort_values(by = "ct", ascending = True).drop_duplicates("channelname", keep = "first")[["channelname","subcnt"]]
#display(answer)

"각채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수 (subcnt) 의 증가량을 구하여라***"
start = channel.loc[channel.ct.dt.strftime("%Y-%m-%d %H") == '2021-10-03 03']
end = channel.loc[channel["ct"].dt.strftime("%Y-%m-%d %H") == '2021-11-01 15']

# 필요한 컬럼만 선택
# concat을 위한 reset_index
start_stack = start[["channelname","subcnt"]].reset_index(drop=True)
end_stack = end[["channelname",'subcnt']].reset_index(drop=True)

# 순서대로 컬럼명을 재 설정 (컬럼 수 맞추기)
# 원래 인덱스 정보는 삭제했으므로 컬럼을 두개만 이용 
start_stack.columns = ["channelname","start_sub"]
end_stack.columns = ["channelname","end_sub"]

#stack = pd.concat([start_stack, end_stack],axis=1)
stack = pd.merge(start_stack, end_stack)    # 중복되는 컬럼은 생략하고 병합
stack["del"] = stack["end_sub"] - stack["start_sub"]

#print(stack[["channelname","del"]].head())  # concat으로 데이터 합치면 모든 channelname 을 출력



"각 에피소드의 시작날짜(년-월-일)를 에피소드 이름과 묶어 데이터 프레임으로 만들고 출력하라"
start_date = video.sort_values(by = "ct", ascending= False).drop_duplicates("videoname", keep = "last")[["videoname","ct"]].reset_index(drop=True)
start_date["date"] = start_date.ct.dt.date
start_date.drop(columns = "ct", inplace = True)
start_date["new_col"] = start_date["date"].astype(str) + start_date["videoname"].astype(str)
answer = start_date[["new_col"]]
#display(answer)

"""공범 컨텐츠의 경우 19:00시에 공개 되는것으로 알려져있다.
 공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 viewcnt를 내림차순으로 정렬하여 출력하라"""

target = video.copy()
target["hour"] = target.ct.dt.hour

target = target.loc[target.hour == 21]
answer = target.sort_values(by = "ct", ascending = True).drop_duplicates("videoname",keep="first").reset_index(drop = True)
answer = answer[["viewcnt","ct","videoname"]]
# display(answer)


""" video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고
videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차순으로 정렬하라"""

stack = video.sort_values(by = "ct" , ascending = True).drop_duplicates("videoname",keep = "last")
stack["ratio"] = stack["dislikecnt"]/stack["likecnt"]
stack = stack[["videoname","ratio"]].sort_values(by = "ratio", ascending = True)
# display(stack)

"""2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오***"""
stack = video.loc[(video.ct.dt.strftime("%Y-%m-%d %hh:%mm:%ss") >= "2021-11-01 00:00:00")&  \
                  (video.ct.dt.strftime("%Y-%m-%d %hh:%mm:%ss") <= "2021-11-01 15:00:00")]
start_view = stack.drop_duplicates("videoname", keep = "first").reset_index(drop=True)
end_view = stack.drop_duplicates("videoname", keep = "last").reset_index(drop=True)

start_view = start_view[["videoname","viewcnt"]]
start_view.columns = ["videoname","start_view_cnt"]

end_view = end_view[["viewcnt"]]
end_view.columns = ["end_view_cnt"]

stack = pd.concat([start_view,end_view],axis = 1)
stack["increse"] = stack["end_view_cnt"] - stack["start_view_cnt"]
#display(stack)

"""video 데이터 중에서 중복되는 데이터가 존재한다. 중복되는 각 데이터의 시간대와 videoname 을 구하여라"""
origin = set(video.index)
remove = set(video.drop_duplicates().index)
#dup_set = origin - remove
stack1 = video.loc[(video.index ==722)]
stack2 = video.loc[(video.index ==3927)]
stack = pd.concat([stack1, stack2])
display(stack)








,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
722,c5JQp6xafqc,2228250,34559,849,0,7191,2021-10-13 09:41:37,공범 EP1
3927,23QQ1ru9YQg,1408005,20484,554,0,8567,2021-10-13 09:41:37,공범 EP2
